In [1]:
print("hello")

hello


In [10]:
import torch
import torch.nn as nn

class ConvNet(nn.Module):

    class Block(nn.Module):
        def __init__(self, in_channels, out_channels, stride):
            super().__init__()
            kernel_size = 3
            padding = (kernel_size - 1) // 2

            self.block_model = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
                nn.Conv2d(out_channels, out_channels, kernel_size, 1, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
                nn.Conv2d(out_channels, out_channels, kernel_size, 1, padding, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
            )

            # Residual connection
            if in_channels != out_channels or stride != 1:
                self.skip = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False)
            else:
                self.skip = nn.Identity()

        def forward(self, x):
            return self.block_model(x) + self.skip(x)

    def __init__(self, channels_l0=64, n_blocks=2, num_classes=8):
        super().__init__()
        cnn_layers = [
            nn.Conv2d(in_channels=3, out_channels=channels_l0, kernel_size=11, stride=2, padding=(11 - 1) // 2, bias=False),
            nn.BatchNorm2d(channels_l0),
            nn.ReLU(),
        ]

        c1 = channels_l0
        for _ in range(n_blocks):
            c2 = c1 * 2
            cnn_layers.append(self.Block(c1, c2, stride=2))
            c1 = c2

        cnn_layers.append(nn.AdaptiveAvgPool2d((1, 1)))  # Global Average Pooling
        self.cnn = nn.Sequential(*cnn_layers)
        self.classifier = nn.Linear(c1, num_classes)

    def forward(self, x):
        x = self.cnn(x)
        x = x.view(x.size(0), -1)  # flatten for the classifier
        x = self.classifier(x)
        return x

In [6]:
!pip install torch.cuda.amp

ERROR: Could not find a version that satisfies the requirement torch.cuda.amp
ERROR: No matching distribution found for torch.cuda.amp


In [16]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm
#from torch.amp import autocast, GradScaler
from torch.utils.data import random_split
from torchvision.transforms import ToTensor

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Paths
data_dir = "Teeth or Dental Image Dataset"  # update this to your local path

# # Transforms
transform = transforms.Compose([
     transforms.Resize((128, 128)),   # Resize to make inputs consistent
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
 ])

# Apply your full transform pipeline
dataset = datasets.ImageFolder(root=data_dir, transform=transform)

# Set seed for reproducibility
torch.manual_seed(42)

# Choose how many images you want
subset_size = 4000

# Split into a subset and discard the rest
subset, _ = random_split(dataset, [subset_size, len(dataset) - subset_size])

# Now split the subset into train/val
train_size = int(0.8 * subset_size)
val_size = subset_size - train_size
train_subset, val_subset = random_split(subset, [train_size, val_size])

# Create dataloaders
train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)

model = ConvNet(channels_l0=32, n_blocks=2, num_classes=8)

In [27]:
# Model, loss, optimizer
model = ConvNet(channels_l0=32, n_blocks=2, num_classes=8).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0.01)

# Initialize GradScaler
scaler = GradScaler(device='cuda')

num_epochs = 15
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_preds = 0
    total = 0

    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        images, labels = images.to(device), labels.to(device)

        # Zero gradients
        optimizer.zero_grad()

        # Mixed precision forward pass
        with autocast(device_type='cuda'):  # Automatically use mixed precision
            outputs = model(images)
            loss = criterion(outputs, labels)

        # Scale the loss and backpropagate
        scaler.scale(loss).backward()

        # Step the optimizer and update the scaler
        scaler.step(optimizer)
        scaler.update()

        # Track running loss and accuracy
        running_loss += loss.item() * images.size(0)
        _, predicted = outputs.max(1)
        correct_preds += predicted.eq(labels).sum().item()
        total += labels.size(0)

    train_accuracy = 100. * correct_preds / total
    avg_loss = running_loss / total

    # Validation
    model.eval()
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            with autocast(device_type='cuda'):  # Ensure mixed precision during validation as well
                outputs = model(images)
            _, predicted = outputs.max(1)
            val_correct += predicted.eq(labels).sum().item()
            val_total += labels.size(0)
    val_accuracy = 100. * val_correct / val_total

    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {avg_loss:.4f} Train Acc: {train_accuracy:.2f}% Val Acc: {val_accuracy:.2f}%")

torch.save(model.state_dict(), "/content/drive/MyDrive/MSAI/AI in Health/HRP/teeth_cnn_model.pth")
print("Model saved as teeth_cnn_model.pth")


Epoch 1/15: 100%|██████████| 100/100 [00:34<00:00,  2.87it/s]


Epoch [1/15] Loss: 1.1251 Train Acc: 59.94% Val Acc: 65.88%


Epoch 2/15: 100%|██████████| 100/100 [00:34<00:00,  2.91it/s]


Epoch [2/15] Loss: 0.5362 Train Acc: 85.50% Val Acc: 74.88%


Epoch 3/15: 100%|██████████| 100/100 [00:34<00:00,  2.87it/s]


...

Epoch [13/15] Loss: 0.0463 Train Acc: 98.53% Val Acc: 97.75%


Epoch 14/15: 100%|██████████| 100/100 [00:34<00:00,  2.91it/s]


Epoch [14/15] Loss: 0.0439 Train Acc: 98.62% Val Acc: 98.25%


Epoch 15/15: 100%|██████████| 100/100 [00:34<00:00,  2.90it/s]


Epoch [15/15] Loss: 0.0292 Train Acc: 99.19% Val Acc: 98.38%
Model saved as teeth_cnn_model.pth


# Model Testing

In [8]:
class_to_idx = dataset.class_to_idx
class_names = []
for class_name, idx in class_to_idx.items():
    class_names.append(class_name)
    print(f"Class: {class_name}, Index: {idx}")

#print("Predicted class label:", class_names[predicted_class])

Class: Lower Front, Index: 0
Class: Lower Left, Index: 1
Class: Lower Occlusal, Index: 2
Class: Lower Right, Index: 3
Class: Upper Front, Index: 4
Class: Upper Left, Index: 5
Class: Upper Occlusal, Index: 6
Class: Upper Right, Index: 7


In [18]:
from PIL import Image
import torchvision.transforms as transforms
import torch
image_path = "/Users/shubhjavia/Desktop/MSAI/SP'25/AI in Health/High Risk Tutorial/LR 1238.jpg"  # Change this
image = Image.open(image_path).convert("RGB")  # Ensure it's 3-channel
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension: shape [1, 3, H, W]
#input_tensor = input_tensor.to(device)  # e.g., device = torch.device("cuda")
model.load_state_dict(torch.load("/Users/shubhjavia/Desktop/MSAI/SP'25/AI in Health/High Risk Tutorial/teeth_cnn_model.pth",map_location=torch.device('cpu')))
model.eval()  # set to eval mode for inference
output = model(input_tensor)
predicted_class = output.argmax(1).item()  # Get predicted class index

print("Predicted class index:", predicted_class)
print("predicted Class Label:",class_names[predicted_class])


Predicted class index: 3
predicted Class Label: Lower Right
